In [1]:
import pandas as pd 
import folium

## Making a web map with folium 

In [2]:
my_map = folium.Map(location = [37.756488877551,-122.429343346939],zoom_start=12,tiles='Stamen Terrain')
folium.Marker([37.756488877551,-122.429343346939], popup='SanFan',icon = folium.Icon(color ='red')).add_to(my_map)
folium.LatLngPopup().add_to(my_map)
folium.ClickForMarker(popup='Waypoint').add_to(my_map)

In [3]:
my_map.save('first_map_test.html')

## Extracting longitudes and latitudes with geopy 

In [4]:
import geopy

In [5]:
from geopy.geocoders import Nominatim
nom = Nominatim()
n= nom.geocode('3666 21st St,San Francisco,CA 94114')
print(n.longitude)
print(n.latitude)

-122.429343346939
37.756488877551


In [6]:
nom.geocode('mansoura,egypt')

Location(المنصورة, الدقهلية, مصر, (31.037566, 31.38649, 0.0))

## Build a web map with a dataframe 

In [7]:
data = pd.read_csv('Volcanoes_USA.txt')

In [8]:
volcanoes_map = folium.Map(location = [37.756488877551,-122.429343346939],zoom_start=5,tiles='Stamen Terrain')

for lat,long,name,elev in zip(data['LAT'],data['LON'],data['NAME'],data['ELEV']):
    folium.Marker([lat,long], popup=name,icon = folium.Icon(color='green' if elev in range(0,1000)\
                                                            else 'orange' if elev in range(1000,3000)\
                                                            else 'red')).add_to(volcanoes_map)
volcanoes_map.save('volcanoes_map.html')

In [9]:
def coloring(elev):
    minimum = int(min(data['ELEV']))
    step = int(max(data['ELEV']-min(data['ELEV']))/3)
    if elev in range(minimum,minimum+step):
        col = 'green'
    elif elev in range(minimum+step,minimum+step*2):
        col = 'orange'
    else :
        col = 'red'
    return col

elevation_list = []
for i in data['ELEV']:
    num = coloring(i)
    elevation_list.append(num)
    
data['elevation'] = elevation_list

data.head()

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON,elevation
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997,red
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000,red
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000,red
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000,orange
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997,red


In [10]:
volcanoes_new = folium.Map(location = [data['LAT'].mean(),data['LON'].mean()],zoom_start=5,tiles='Stamen Terrain')

for lat,long,name,elev in zip(data['LAT'],data['LON'],data['NAME'],data['elevation']):
    folium.Marker([lat,long], popup=name,icon = folium.Icon(color=elev,icon_color='black')).add_to(volcanoes_new)
volcanoes_new.save('volcanoes_new.html')

In [11]:
data['STATUS'].unique()

array(['Historical', 'Tephrochronology', 'Dendrochronology', 'Radiocarbon',
       'Varve Count', 'Holocene?', 'Holocene', 'Pleistocene-Fumarolic',
       'Anthropology'], dtype=object)

## To convert shape file to gepjson files online : http://ogre.adc4gis.com/
To download geojspn converter : http://www.qgis.org/en/site/forusers/download.html

In [22]:
world_population = folium.Map(location = [data['LAT'].mean(),data['LON'].mean()],zoom_start=5,tiles='Mapbox bright')

#Add layers to a feature group variable 
fg = folium.FeatureGroup(name='Volcano Locations')    

for lat,long,name,elev in zip(data['LAT'],data['LON'],data['NAME'],data['elevation']):
    fg.add_child(folium.Marker(location=[lat,long], popup=name,icon = folium.Icon(color=elev,icon_color='black')))

#add the feature group to the map 
world_population.add_child(fg)
world_population.add_child(folium.GeoJson(data=open('shape_file.json'),name='World Population',\
                           style_function= lambda x : {'fillColor':'green' if x['properties']['POP2005']<=10000000 \
                                                       else 'orange' if 10000000 < x['properties']['POP2005']<=20000000\
                                                      else 'red'}))
world_population.add_child(folium.LayerControl())
world_population.save('world_population.html')
